# Imports

In [ ]:
from __future__ import print_function
from __future__ import division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import glob
import shutil
from PIL import Image

from src.pytorch_utils import train_model, predict

# Prerequisites

Before training the model, one must have the following structure in the DATA_DIR : \
![tree](directory_structure.png)

In [ ]:
DATA_DIR = "data/spectogram/"

# Training

In [ ]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load model
model = models.resnet18(pretrained=True) # or false 

In [ ]:
# Freeze weights if transfer learning
for param in model.parameters():
    param.requires_grad = False

# Update last layer to have correct output layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

In [ ]:
# Check the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

In [ ]:
# To GPU (if possible)
model = model.to(device)

In [ ]:
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

batch_size = 32

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x]) for x in ['train', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'test']}

In [ ]:
class_names = image_datasets['train'].classes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
trained_model = train_model(model, dataloaders_dict, criterion, optimizer, exp_lr_scheduler, num_epochs=20)

In [ ]:
# Save Model
torch.save(trained_model.state_dict(), "Resnet_SGD_valscore_58.pt")

# Predict 

In [ ]:
# Load Model
# First load the original one
model = models.resnet18().to(device)

# Change the last layer shape
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Put trained weights
model.load_state_dict(torch.load("Resnet_SGD_valscore_60.pt",map_location=torch.device(device)))

In [ ]:
class_names = ["angry", "happy", "relaxed", "sad"]

In [ ]:
# Follow the same transformation steps
loader = transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
categs = ["angry", "happy", "relaxed", "sad"]
true = []
pred = []
for categ in categs:
    files = glob.glob("../data/spectogram_old/test/" + categ+ "/*.jpg")
    for file in files:
        proba, val = predict(model, file, loader, class_names, k=1)
        true.append(categ)
        pred.append(val[0])

In [ ]:
df = pd.DataFrame({"true": true, "pred":pred})

In [ ]:
pd.crosstab(df.true, df.pred)